In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
from docx import Document
import os
from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import pdfplumber
import nltk
nltk.download('punkt')
from textblob import TextBlob
import plotly.graph_objects as go
import plotly.express as px
import os
import json
from docx import Document
import pdfplumber
from bs4 import BeautifulSoup
import spacy
import matplotlib.pyplot as plt
from wordcloud import WordCloud
spacy.cli.download("en_core_web_sm")
import pandas as pd
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from collections import Counter
from plotly.offline import plot
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/nicholasreese/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 1.0 MB/s eta 0:00:0000:0100:01m
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [2]:
#!pip install transformers torch

In [3]:
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification, pipeline

/Users/nicholasreese/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:260: FutureWarning:

`torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.



***

## Loading the Model and Tokenizer

In [4]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels = 2)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
resume_text = """

Nicholas E. Reese
Van Ness, Washington D.C.
412-216-2398
Nicholas.E.Reese15@gmail.com 
Education 
•Georgetown University - August 2023 – December 2024
Master of Science Business Analytics – MSBA candidate
Peer-Elected Class Representative 
•Dickinson College - August 2014 – September 2018
Economics & Political Science Double Major
Varsity Tennis Captain 
•University of Bologna - August 2016

Skills
- R studio
- Python
- Power BI
- SQL
- Neural Networks
- Machine Learning
- Macro Modeling
- AWS Cloud Services
- Pandas
- A/B Testing
- Scikit-Learn
- Econometrics

Professional Experience
FINRA, Washington, D.C.	June 2020 – Present 
Senior Analyst, Market Regulation	September 2022 – Present 
•Earned top 10% of performance of analysts for past two years.
•Led team in research implementing PostgreSQL and NoSQL queries for large data pulls.
•Spearheaded new analytical approaches to financial workflow with tools such as R & Python for model development.
•Developed working predictive modeling schemas for senior staff using statistical analysis.
•Leveraged skills in platforms like Python, R, Power BI, Tableau and SQL accompanied with strong statistical background in financial markets.
•Built the Security-Based Swap training manual deck & produced the recorded info session for all of FINRA. 
•Selected for the FINRA’s first Georgetown Advanced Analytics Program as one of the most junior staff awarded opportunity.
•Incorporated statistical analytics to assist in creating a NPL tool to analyze financial documents language to minimize time on manual analysis.
•Produced unsupervised and supervised models to perform analysis for Security-Based Swaps trade patterns.
•Improved FINRA platforms with Data Scientists for higher accuracy and efficiency. 
•Managed process of re-engineering supervisory reviews through advanced analytic tools for senior staff.
•Implements advanced analytics to assist senior staff with maximizing efficiencies in daily workloads and trade pattern creations.
•Presented visualization case work using Power BI & Tableau to senior leadership.
•Persuaded senior staff to allow for statistical analytics tools like R.
•Mentored over 20 junior & senior staff members on improving processes with analytical tools for financial reviews.
•Managed junior staff with day-to-day workload while teaching the staff how to use advanced analytics.
Analyst, Market Regulation	June 2021-September 2022
·Created macroeconomic models for newly implemented FINRA Rule 2232 and MSRB Rule G-15.
·Mentored junior staff with creating macro models, synthesizing responses from FINRA member firms.
·Won the Regulator Scholarship for continued financial learning based on individual performance.
·Received six internal awards for expertise in Municipal and Corporate Bond analysis as an analyst.
Associate Analyst, Market Regulation	June 2020 – June 2021
·Created macroeconomic models to quantify business models of selected firms for FINRA Rule 2232.
             BNY Mellon, Pittsburgh, PA						                 September 2019 – June 2020
Corporate Trust Associate
·Leader & instructor of the Bloomberg software for the Corporate Trust Team.
LendingHome, Internship, Pittsburgh, PA
Funding Specialist & Post Closing Member	March 2019 – August 2019
·Reviewed closing documents to ensure precise execution for funding staff.
              Veraction/Trax, Junior Business Analyst, Memphis, TN	  June 2016 - August 2016                                                    June 2016 - August 2016
·Led the creation, development, and implementation of Request For Proposal (RFP) database.
"""


In [6]:
#resume_text

***

In [7]:
from transformers import pipeline

In [8]:
ner_pipeline = pipeline('ner', model = "dbmdz/bert-large-cased-finetuned-conll03-english",
                       aggregation_strategy = 'simple')

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [9]:
entities = ner_pipeline(resume_text)

In [10]:
for entity in entities:
    print(f"Entity: {entity['word']}, Label: {entity['entity_group']}, Score: {entity['score']:.4f}")

Entity: Nicholas E, Label: PER, Score: 0.8347
Entity: Reese, Label: PER, Score: 0.8619
Entity: Van Ness, Label: LOC, Score: 0.6532
Entity: Washington D, Label: LOC, Score: 0.9981
Entity: C, Label: LOC, Score: 0.9990
Entity: Nicholas, Label: PER, Score: 0.9735
Entity: E, Label: PER, Score: 0.6905
Entity: Reese, Label: PER, Score: 0.9786
Entity: Georgetown University, Label: ORG, Score: 0.9974
Entity: Science, Label: MISC, Score: 0.7188
Entity: Business, Label: ORG, Score: 0.3938
Entity: ##tics, Label: MISC, Score: 0.6904
Entity: MS, Label: MISC, Score: 0.6772
Entity: ##BA, Label: ORG, Score: 0.6136
Entity: Dickinson College, Label: ORG, Score: 0.9974
Entity: & Political, Label: ORG, Score: 0.6406
Entity: Science, Label: MISC, Score: 0.6226
Entity: University of Bologna, Label: ORG, Score: 0.9910
Entity: Python, Label: MISC, Score: 0.7331
Entity: B, Label: MISC, Score: 0.3749
Entity: S, Label: MISC, Score: 0.4090
Entity: N, Label: MISC, Score: 0.3621
Entity: Networks, Label: MISC, Score:

In [11]:
filtered_entities = [entity for entity in entities if entity['entity_group'] != 'MISC']

In [12]:
for entity in filtered_entities:
    print(f"Entity: {entity['word']}, Label: {entity['entity_group']}, Score: {entity['score']:.4f}")

Entity: Nicholas E, Label: PER, Score: 0.8347
Entity: Reese, Label: PER, Score: 0.8619
Entity: Van Ness, Label: LOC, Score: 0.6532
Entity: Washington D, Label: LOC, Score: 0.9981
Entity: C, Label: LOC, Score: 0.9990
Entity: Nicholas, Label: PER, Score: 0.9735
Entity: E, Label: PER, Score: 0.6905
Entity: Reese, Label: PER, Score: 0.9786
Entity: Georgetown University, Label: ORG, Score: 0.9974
Entity: Business, Label: ORG, Score: 0.3938
Entity: ##BA, Label: ORG, Score: 0.6136
Entity: Dickinson College, Label: ORG, Score: 0.9974
Entity: & Political, Label: ORG, Score: 0.6406
Entity: University of Bologna, Label: ORG, Score: 0.9910
Entity: AWS, Label: ORG, Score: 0.8358
Entity: FINRA, Label: ORG, Score: 0.9555
Entity: Washington, Label: LOC, Score: 0.9981
Entity: D, Label: LOC, Score: 0.9980
Entity: C, Label: LOC, Score: 0.9992
Entity: Regulation, Label: ORG, Score: 0.4722
Entity: Power, Label: ORG, Score: 0.4565
Entity: Table, Label: ORG, Score: 0.5572
Entity: FINRA, Label: ORG, Score: 0.

In [13]:
import re

# Regex pattern for email extraction
email_pattern = r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}'

# Find all email addresses in the text
emails = re.findall(email_pattern, resume_text)

# Display found email addresses
print(emails)

['Nicholas.E.Reese15@gmail.com']


In [14]:
# Load the NER pipeline
ner_pipeline2 = pipeline("ner", model="dbmdz/bert-large-cased-finetuned-conll03-english", aggregation_strategy="simple")


# Detect entities using the NER pipeline
entities2 = ner_pipeline2(resume_text)

# Regex pattern for email extraction
email_pattern = r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}'

# Find all email addresses in the text
emails = re.findall(email_pattern, resume_text)

# Print detected entities from NER
for entity in entities2:
    print(f"Entity: {entity['word']}, Label: {entity['entity_group']}, Score: {entity['score']:.4f}")



Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Entity: Nicholas E, Label: PER, Score: 0.8347
Entity: Reese, Label: PER, Score: 0.8619
Entity: Van Ness, Label: LOC, Score: 0.6532
Entity: Washington D, Label: LOC, Score: 0.9981
Entity: C, Label: LOC, Score: 0.9990
Entity: Nicholas, Label: PER, Score: 0.9735
Entity: E, Label: PER, Score: 0.6905
Entity: Reese, Label: PER, Score: 0.9786
Entity: Georgetown University, Label: ORG, Score: 0.9974
Entity: Science, Label: MISC, Score: 0.7188
Entity: Business, Label: ORG, Score: 0.3938
Entity: ##tics, Label: MISC, Score: 0.6904
Entity: MS, Label: MISC, Score: 0.6772
Entity: ##BA, Label: ORG, Score: 0.6136
Entity: Dickinson College, Label: ORG, Score: 0.9974
Entity: & Political, Label: ORG, Score: 0.6406
Entity: Science, Label: MISC, Score: 0.6226
Entity: University of Bologna, Label: ORG, Score: 0.9910
Entity: Python, Label: MISC, Score: 0.7331
Entity: B, Label: MISC, Score: 0.3749
Entity: S, Label: MISC, Score: 0.4090
Entity: N, Label: MISC, Score: 0.3621
Entity: Networks, Label: MISC, Score:

***

## Test

In [15]:
resume_test = """
Nicholas E. Reese
Van Ness, Washington D.C.
412-216-2398
Nicholas.E.Reese@gmail.com
"""



In [16]:
entities = ner_pipeline(resume_test)

In [17]:
for entity in entities:
    print(f"Entity: {entity['word']}, Label: {entity['entity_group']}, Score: {entity['score']:.4f}")

Entity: Nicholas E. Reese Van Ness, Label: PER, Score: 0.8019
Entity: Washington D, Label: LOC, Score: 0.9986
Entity: C, Label: LOC, Score: 0.9995
Entity: Nicholas, Label: PER, Score: 0.9980
Entity: E. Reese, Label: PER, Score: 0.9686


In [18]:
thresholds ={
    'PER': .7,
    'ORG': .8
}

In [19]:
redacted_text = resume_test

In [20]:
for entity in entities:
    entity_type = entity['entity_group']
    score = entity['score']
    
    if entity_type == entity and score >= thresholds[entity]:
        entity_text = entity['word']
        redacted_text = re.sub(re.escape(entity_text), '[Redacted]', redacted_text)

# Redact the email addresses
redacted_text = re.sub(r'\S+@\S+', '[Redacted Email]', redacted_text)



print(redacted_text)


Nicholas E. Reese
Van Ness, Washington D.C.
412-216-2398
[Redacted Email]



In [21]:
#print(redacted_text)

In [22]:
resume_test = """
Nicholas E. Reese
Van Ness, Washington D.C.
412-216-2398
Nicholas.E.Reese@gmail.com
"""



In [23]:
entities = ner_pipeline(resume_test)

In [24]:
for entity in entities:
    print(f"Entity: {entity['word']}, Label: {entity['entity_group']}, Score: {entity['score']:.4f}")

Entity: Nicholas E. Reese Van Ness, Label: PER, Score: 0.8019
Entity: Washington D, Label: LOC, Score: 0.9986
Entity: C, Label: LOC, Score: 0.9995
Entity: Nicholas, Label: PER, Score: 0.9980
Entity: E. Reese, Label: PER, Score: 0.9686


In [25]:
thresholds ={
    'PER': .8,
    'ORG': .9,
    'LOC': .8
}

In [26]:
redacted_text = resume_test

In [27]:
entities_sorted = sorted(entities, key=lambda x: len(x['word']), reverse=True)


for entity in entities_sorted:
    entity_type = entity['entity_group']
    score = entity['score']
    
    if entity_type == entity_type and score >= thresholds[entity_type]:
        entity_text = entity['word']
        redacted_text = re.sub(re.escape(entity_text), '[Redacted]', redacted_text)


        
for entity in entities_sorted:
    entity_type = entity['entity_group']
    score = entity['score']
    
    if entity_type in thresholds and score >= thresholds[entity_type]:
        entity_text = entity['word']
        # Make sure to redact the smaller parts again if needed
        redacted_text = re.sub(re.escape(entity_text), '[Redacted]', redacted_text)

# Redact the email addresses and phone numbers 
redacted_text = re.sub(r'\S+@\S+', '[Redacted Email]', redacted_text)
redacted_text = re.sub(r'\(?\d{3}\)?[-.\s]?\d{3}[-.\s]?\d{4}', '[Redacted Phone]', redacted_text)



In [28]:
print(redacted_text)


[Redacted] [Redacted]
Van Ness, [Redacted].[Redacted].
[Redacted Phone]
[Redacted Email]



***

## Adding In Resumes 

In [32]:
import os
import json
from docx import Document
import pdfplumber

def convert_files_to_json(folder_path, output_json_file):
    corpus = []
    
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)

        # Skip temporary files created by Word
        if filename.startswith('~$'):
            print(f"Skipped temporary file: {filename}")
            continue
        
        if filename.endswith('.docx'):
            try:
                doc = Document(file_path)
                text = '\n'.join([para.text for para in doc.paragraphs])
                document_data = {
                    'title': filename,
                    'text': text, 
                    'type': 'word',
                    'file_path': file_path
                }
                corpus.append(document_data)
                print(f"Added .docx: {filename}") 
        
            except Exception as e:
                print(f"Error processing .docx file {filename}: {e}")
        
        elif filename.endswith('.pdf'):
            try:
                with pdfplumber.open(file_path) as pdf:
                    text = '\n'.join([page.extract_text() for page in pdf.pages if page.extract_text()])
                    document_data = {
                        'title': filename, 
                        'text': text,
                        'type': 'pdf',
                        'file_path': file_path
                    }
                    corpus.append(document_data)
                    print(f"Added .pdf: {filename}")
                    
            except Exception as e:
                print(f"Error processing .pdf file {filename}: {e}")
                
    with open(output_json_file, 'w', encoding='utf-8') as json_file:
        json.dump(corpus, json_file, ensure_ascii=False, indent=4)

# Call the function with your paths
convert_files_to_json('/Users/nicholasreese/Desktop/Georgetown/Capstone/capstone_github/Capstone_Introduction/Saxa-4-Capstone', 'output_corpus.json')
# change your file path to where you saved the resumes

Added .docx: Genesis Roberto 2024 Resume.docx
Added .docx: Jonathan J Saville resume.docx
Added .pdf: Dezmond Richardson GU Q3.2024 Resume.docx.pdf
Added .docx: Nicholas Reese Resume .docx


In [33]:
resumes = pd.read_json('output_corpus.json')

In [34]:
resumes = pd.DataFrame(resumes)

In [35]:
resumes

,title,text,type,file_path
0,Genesis Roberto 2024 Resume.docx,"Ms. Genesis U. Roberto \n Rockville, MD ...",word,/Users/nicholasreese/Desktop/Georgetown/Capsto...
1,Jonathan J Saville resume.docx,"Jonathan J Saville\n503 Edwards Ave, Apt #7 ...",word,/Users/nicholasreese/Desktop/Georgetown/Capsto...
2,Dezmond Richardson GU Q3.2024 Resume.docx.pdf,D R\nEZMOND ICHARDSON\nddr34@georgetown.edu ▪ ...,pdf,/Users/nicholasreese/Desktop/Georgetown/Capsto...
3,Nicholas Reese Resume .docx,"Nicholas E. Reese\nVan Ness, Washington D.C.\n...",word,/Users/nicholasreese/Desktop/Georgetown/Capsto...


In [38]:
resumes = resumes.drop('file_path', axis = 1)

In [39]:
resumes

,title,text,type
0,Genesis Roberto 2024 Resume.docx,"Ms. Genesis U. Roberto \n Rockville, MD ...",word
1,Jonathan J Saville resume.docx,"Jonathan J Saville\n503 Edwards Ave, Apt #7 ...",word
2,Dezmond Richardson GU Q3.2024 Resume.docx.pdf,D R\nEZMOND ICHARDSON\nddr34@georgetown.edu ▪ ...,pdf
3,Nicholas Reese Resume .docx,"Nicholas E. Reese\nVan Ness, Washington D.C.\n...",word


In [40]:
from transformers import pipeline

In [41]:
ner_pipeline = pipeline('ner', model = "dbmdz/bert-large-cased-finetuned-conll03-english",
                       aggregation_strategy = 'simple')

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [47]:
resumes['text_list'] = resumes['text'].tolist()

In [53]:
resumes['clean_text'] = resumes['text'].fillna("").astype(str)

resume_text = resumes['clean_text'].tolist()

entities = ner_pipeline(resume_text)

In [58]:
resumes = resumes.drop('text_clean', axis = 1)
resumes = resumes.drop('text_list', axis = 1)

In [59]:
resumes

,title,text,type,clean_text
0,Genesis Roberto 2024 Resume.docx,"Ms. Genesis U. Roberto \n Rockville, MD ...",word,"Ms. Genesis U. Roberto \n Rockville, MD ..."
1,Jonathan J Saville resume.docx,"Jonathan J Saville\n503 Edwards Ave, Apt #7 ...",word,"Jonathan J Saville\n503 Edwards Ave, Apt #7 ..."
2,Dezmond Richardson GU Q3.2024 Resume.docx.pdf,D R\nEZMOND ICHARDSON\nddr34@georgetown.edu ▪ ...,pdf,D R\nEZMOND ICHARDSON\nddr34@georgetown.edu ▪ ...
3,Nicholas Reese Resume .docx,"Nicholas E. Reese\nVan Ness, Washington D.C.\n...",word,"Nicholas E. Reese\nVan Ness, Washington D.C.\n..."


In [62]:
from transformers import pipeline

ner_pipeline = pipeline('ner', model = "dbmdz/bert-large-cased-finetuned-conll03-english",
                       aggregation_strategy = 'simple')

#entities = ner_pipeline(resume_text)

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [65]:
for doc_entities in entities:
    for entity in doc_entities:
        print(f"Entity: {entity['word']}, Label: {entity['entity_group']}, Score: {entity['score']:.4f}")

Entity: Genesis U. Roberto Rockville, Label: PER, Score: 0.9323
Entity: MD, Label: LOC, Score: 0.9810
Entity: MD, Label: LOC, Score: 0.9903
Entity: VA, Label: LOC, Score: 0.9143
Entity: US GAAP, Label: MISC, Score: 0.7830
Entity: SOX, Label: ORG, Score: 0.8475
Entity: M & A, Label: ORG, Score: 0.8706
Entity: Master, Label: MISC, Score: 0.4755
Entity: Analysis, Label: MISC, Score: 0.4144
Entity: Georgetown University, Label: ORG, Score: 0.9939
Entity: Saggar, Label: PER, Score: 0.9326
Entity: Rosenberg, Label: PER, Score: 0.9968
Entity: Rockville, Label: LOC, Score: 0.9400
Entity: MD, Label: LOC, Score: 0.9427
Entity: Audit Services, Label: ORG, Score: 0.8769
Entity: ASC, Label: MISC, Score: 0.6365
Entity: Ernst and Young LLP, Label: ORG, Score: 0.9961
Entity: Tysons, Label: LOC, Score: 0.9479
Entity: Virginia, Label: LOC, Score: 0.9918
Entity: Strategy and Transactions, Label: ORG, Score: 0.8156
Entity: & A, Label: ORG, Score: 0.8782
Entity: Alteryx, Label: ORG, Score: 0.8925
Entity: P

In [ ]:
#email_data = pd.read_csv('emails.csv')

In [ ]:
#email_data = pd.DataFrame(email_data)

#email_data.to_csv('emails.csv', index=False, encoding='utf-8')

In [ ]:
#email_data

In [ ]:
#from datasets import Dataset

In [ ]:
#dataset = Dataset.from_pandas(email_data)

In [ ]:
#email_data = Dataset.from_pandas(email_data)